In [120]:
import json
import pandas as pd
import numpy as np
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps
import simpler_sf
simpler_sf.simple_salesforce()
from sqlalchemy import create_engine

In [121]:
with open("../login.json", "r") as j:
    misDatos = json.load(j)
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [122]:
ruts = "F:/martin/MetLife SFTP/ORIENTA_22122023.xlsx"
rutsMetlife2 = pd.read_excel(ruts)


In [123]:
rutsMetlife2.head()


,POLIZA,RUTASEGURADO,DV,RUTBENEFICIARIO,RUTBENEFIARIO_DV,FNAC,SEXO,DIRECCION,COMUNA,CIUDAD,TELEFONO,RELACION,DESDE,HASTA,NOMBRE,EMAIL,NROCELULAR,VIGENCIA
0,340002977.0,26253599,1,26253599.0,1,1981-06-29 00:00:00,F,NaN,NaN,NaN,961648617,AS,2023-06-01,NaN,Meza Patio Patricia,meza.paty@gmail.com,961648617,2024-08-31 00:00:00
1,340002977.0,26346168,1,26346168.0,1,1991-09-01 00:00:00,M,Hipódromo Chile 1310 apto 401,INDEPENDENCIA,SANTIAGO,985606814,AS,2018-03-01,NaN,Pimienta Pardo Danny Daniel,danny.pimienta@cummins.cl,985606814,2024-08-31 00:00:00
2,340002977.0,26346168,1,500000.0,9,1991-07-23 00:00:00,F,Hipódromo Chile 1310 apto 401,INDEPENDENCIA,SANTIAGO,985606814,CO,2019-04-01,NaN,Martinez Gonzalez Estefania Paola,danny.pimienta@cummins.cl,985606814,NaN
3,340002977.0,26370990,K,26370990.0,K,1991-07-31 00:00:00,F,Carmen 566,Santiago centro,Santiago,950709724,AS,2020-03-01,NaN,Argote Argote Keysi Milu,keysi.argote@cummins.cl,950709724,2024-08-31 00:00:00
4,340002977.0,26370990,K,28308780.0,8,2023-12-11 00:00:00,F,Carmen 566,Santiago centro,Santiago,950709724,HI,2023-12-11,NaN,Argote Argote Elif Monserrat,keysi.argote@cummins.cl,950709724,NaN


In [124]:
rutsMetlife = rutsMetlife2[
    [
        "POLIZA",
        "RUTASEGURADO",
        "DV",
        "RUTBENEFICIARIO",
        "RUTBENEFIARIO_DV",
        "RELACION",
    ]
]
rutsMetlife.shape

(811694, 6)

In [125]:
def siPrimerCero(y):
    if len(y) == 2:
        if y[0] == "0":
            y = y[1:]
    return y

In [126]:
rutsMetlife.head()

,POLIZA,RUTASEGURADO,DV,RUTBENEFICIARIO,RUTBENEFIARIO_DV,RELACION
0,340002977.0,26253599,1,26253599.0,1,AS
1,340002977.0,26346168,1,26346168.0,1,AS
2,340002977.0,26346168,1,500000.0,9,CO
3,340002977.0,26370990,K,26370990.0,K,AS
4,340002977.0,26370990,K,28308780.0,8,HI


In [127]:

rutsMetlife["RUTBENEFIARIO_DV"] = rutsMetlife["RUTBENEFIARIO_DV"].astype(str).astype(str).apply(lambda y: siPrimerCero(y))
rutsMetlife.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_1080\3045764418.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rutsMetlife["RUTBENEFIARIO_DV"] = rutsMetlife["RUTBENEFIARIO_DV"].astype(str).astype(str).apply(lambda y: siPrimerCero(y))


,POLIZA,RUTASEGURADO,DV,RUTBENEFICIARIO,RUTBENEFIARIO_DV,RELACION
0,340002977.0,26253599,1,26253599.0,1,AS
1,340002977.0,26346168,1,26346168.0,1,AS
2,340002977.0,26346168,1,500000.0,9,CO
3,340002977.0,26370990,K,26370990.0,K,AS
4,340002977.0,26370990,K,28308780.0,8,HI


In [128]:

rutsMetlife["Relación"] = rutsMetlife["RELACION"].astype(str)
rutsMetlife["Relación"] = np.where(rutsMetlife["Relación"]== "AS","Titular", "Carga")
rutsMetlife = rutsMetlife[rutsMetlife["Relación"]=="Titular" ]

rutsMetlife["Poliza"] = rutsMetlife["POLIZA"].astype(int).astype(str)
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace("-", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace("–", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].astype(str)
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace(".", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace(" ", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.upper()
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.strip()

rutsMetlife["dv_asegurado"] = rutsMetlife["DV"].astype(str).apply(lambda y: siPrimerCero(y)) 
#rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].astype(str)
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.replace("-", "")
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.replace("–", "")
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].astype(str)
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.replace(".", "")
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.replace(" ", "")
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.upper()
rutsMetlife["dv_asegurado"] = rutsMetlife["dv_asegurado"].str.strip()

rutsMetlife["Rut_Titular"] = rutsMetlife["RUTASEGURADO"].astype(str).apply(lambda y: siPrimerCero(y)) 
#rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].astype(str).apply(lambda y: siPrimerCero(y)) 
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace("-", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace("–", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].astype(str)
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace(".", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace(" ", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.upper()
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.strip()

rutsMetlife["dv_Beneficiario"] = rutsMetlife["RUTBENEFIARIO_DV"].astype(str).apply(lambda y: siPrimerCero(y))
#rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].astype(str)
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.replace("-", "")
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.replace("–", "")
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].astype(str)
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.replace(".", "")
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.replace(" ", "")
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.upper()
rutsMetlife["dv_Beneficiario"] = rutsMetlife["dv_Beneficiario"].str.strip()

rutsMetlife["Rut_Beneficiario"] = rutsMetlife["RUTBENEFICIARIO"].astype(int).astype(str).apply(lambda y: siPrimerCero(y))
#rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].astype(str)
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace("-", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace("–", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].astype(str)
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace(".", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace(" ", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.upper()
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.strip()

rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"] + rutsMetlife["dv_Beneficiario"]
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"] + rutsMetlife["dv_asegurado"]


#Subir_pacientes[Subir_pacientes["RutEstaoNo"] == True]


#rutsMetlife["llave"] = rutsMetlife["Poliza"] + rutsMetlife["Rut_Titular"] + rutsMetlife["Rut_Beneficiario"]
rutsMetlife["llave"] = rutsMetlife["Rut_Titular"] + rutsMetlife["Rut_Beneficiario"]

""" rutsMetlife = rutsMetlife[
    ["Poliza",	"Rut_Titular",	"Rut_Beneficiario",	"Relación","llave",
       
    ]
] """
rutsMetlife = rutsMetlife.drop_duplicates(
    [
        "llave",	
    ],
    keep="last",
)
rutsMetlife.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_1080\683568310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rutsMetlife["Relación"] = rutsMetlife["RELACION"].astype(str)
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_1080\683568310.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rutsMetlife["Relación"] = np.where(rutsMetlife["Relación"]== "AS","Titular", "Carga")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_1080\683568310.py:9: FutureWarning: The default value of regex will change from True t

(334344, 13)

In [129]:
rutsMetlife.head()

,POLIZA,RUTASEGURADO,DV,RUTBENEFICIARIO,RUTBENEFIARIO_DV,RELACION,Relación,Poliza,dv_asegurado,Rut_Titular,dv_Beneficiario,Rut_Beneficiario,llave
40,340003298.0,8321693,K,8321693.0,K,AS,Titular,340003298,K,8321693K,K,8321693K,8321693K8321693K
44,340003298.0,11575425,4,11575425.0,4,AS,Titular,340003298,4,115754254,4,115754254,115754254115754254
46,340003298.0,12791821,K,12791821.0,K,AS,Titular,340003298,K,12791821K,K,12791821K,12791821K12791821K
2946,340002976.0,15979959,K,15979959.0,K,AS,Titular,340002976,K,15979959K,K,15979959K,15979959K15979959K
2951,340003298.0,4108674,2,4108674.0,2,AS,Titular,340003298,2,41086742,2,41086742,4108674241086742


In [130]:

#ruts_metlife = rutsMetlife
""" rutsMetlife = rutsMetlife.sort_values(by="llave", ascending = True)
rutsMetlife.to_excel("F:/martin/MetLife SFTP/Titulares unicos Metlife.xlsx", index=False) """

' rutsMetlife = rutsMetlife.sort_values(by="llave", ascending = True)\nrutsMetlife.to_excel("F:/martin/MetLife SFTP/Titulares unicos Metlife.xlsx", index=False) '

In [131]:
""" query_empresacontactoconvenio = (
    "select N_Documento_Titular__c,Paciente__r.IdentificationId__pc from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = 'MetLife' and Relaci_n_Con_Asegurado__c = 'Titular' "
    
) """


#and Convenio_Activo__c = true 
#empresacontactoconvenio = sf.smart_query(query_empresacontactoconvenio)
#empresacontactoconvenio.

empresacontactoconvenio.head()

,N_Documento_Titular__c,Paciente__r.IdentificationId__pc,llave,Esta
24,196857265,196857265,196857265196857265,False
37,133393641,133393641,133393641133393641,False
69,81122970,81122970,8112297081122970,False
70,181215232,181215232,181215232181215232,False
71,160280859,160280859,160280859160280859,False


In [132]:
ruts = "F:/martin/MetLife SFTP/ORIENTA_22122023.xlsx"
empresacontactoconvenio = pd.read_excel(ruts)

In [ ]:
empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].astype(str)
empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].str.replace("-", "")
empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].str.replace("–", "")
empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].astype(str)
empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].str.replace(".", "")
empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].str.replace(" ", "")
empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].str.upper()
empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].str.strip()

empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].astype(str)
empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].str.replace("-", "")
empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].str.replace("–", "")
empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].astype(str)
empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].str.replace(".", "")
empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].str.replace(" ", "")
empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].str.upper()
empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].str.strip()

empresacontactoconvenio.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_1080\3834120504.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresacontactoconvenio["N_Documento_Titular__c"] = empresacontactoconvenio["N_Documento_Titular__c"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_1080\3834120504.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresacontactoconvenio["Paciente__r.IdentificationId__pc"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"].str.replace(".", "")


,N_Documento_Titular__c,Paciente__r.IdentificationId__pc
0,199175742,199175742
1,15638903K,15638903K
2,272172277,272172277
3,197504749,197504749
4,188405010,188405010


In [ ]:
empresacontactoconvenio["llave"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"] + empresacontactoconvenio["N_Documento_Titular__c"]
empresacontactoconvenio = empresacontactoconvenio.drop_duplicates(
    [
        "llave",	
    ],
    keep="last",
)
empresacontactoconvenio.shape

(372546, 3)

In [ ]:
empresacontactoconvenio["Esta"] = empresacontactoconvenio["llave"].isin(rutsMetlife["llave"])
empresacontactoconvenio = empresacontactoconvenio[empresacontactoconvenio["Esta"]==False]
#rutsMetlife = rutsMetlife[rutsMetlife["Relación"]=="Titular" ]
empresacontactoconvenio.shape

(57233, 4)

In [ ]:

#empresacontactoconvenio.to_excel("F:/martin/MetLife SFTP/diferencia_57.xlsx", index=False) 

In [ ]:
""" ruts_metlife = pd.concat([ruts_metlife, rutsMetlife], axis=0)
ruts_metlife = ruts_metlife.drop_duplicates(
    [
        "Poliza","Rut_Titular",	"Rut_Beneficiario",	"Relación",
    ],
    keep="last",
)
ruts_metlife.shape """

' ruts_metlife = pd.concat([ruts_metlife, rutsMetlife], axis=0)\nruts_metlife = ruts_metlife.drop_duplicates(\n    [\n        "Poliza","Rut_Titular",\t"Rut_Beneficiario",\t"Relación",\n    ],\n    keep="last",\n)\nruts_metlife.shape '

In [ ]:

""" reportes_metlife = "C:/Users/jmartinez/Documents/Metlife_derty.xlsx"
reportes_metlife_fredy = pd.read_excel(reportes_metlife) """

' reportes_metlife = "C:/Users/jmartinez/Documents/Metlife_derty.xlsx"\nreportes_metlife_fredy = pd.read_excel(reportes_metlife) '

In [ ]:
""" reportes_metlife_fredy["llave"] =  reportes_metlife_fredy["Rut"]
ruts_metlife["LLave"] = ruts_metlife["Rut_Beneficiario"]  """

' reportes_metlife_fredy["llave"] =  reportes_metlife_fredy["Rut"]\nruts_metlife["LLave"] = ruts_metlife["Rut_Beneficiario"]  '

In [ ]:
""" reportes_metlife_fredy = pd.merge(
    left=reportes_metlife_fredy,
    right=ruts_metlife,
    how="left",
    left_on="llave",
    right_on="LLave",
) """

' reportes_metlife_fredy = pd.merge(\n    left=reportes_metlife_fredy,\n    right=ruts_metlife,\n    how="left",\n    left_on="llave",\n    right_on="LLave",\n) '

In [ ]:
""" reportes_metlife_fredy.head(5) """

' reportes_metlife_fredy.head(5) '

In [ ]:
#reportes_metlife_fredy.to_excel("C:/Users/jmartinez/Documents/reportes_metlife2.xlsx", index=False)